# xVertSeg Data Generation

For training and evaluation of a binary image segmentation algorithm, we want input of size mxnxnxnx1 and output of size (m, n, n, n, 1).  1 channel.  m examples.  3d volume.  For semantic image segmentation, we want output of size (m, n, n, n, 6) for the one-hot encoded vector of output categories.  xVertSeg also has 2 categorical outcomes.

It would be nice to configure the datagen to sample multiple patches from an image.  That might speed training.

## Imports and Constants, etc.

In [ ]:
import datetime
import importlib
import keras
from keras.layers import (Dense, SimpleRNN, Input, Conv1D, 
                          LSTM, GRU, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D,
                          Conv3D, UpSampling3D, BatchNormalization, Concatenate, Add,
                          GaussianNoise, Dropout
                         )
from keras.models import Model
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import projd
import random
import re
import scipy
import shutil
import SimpleITK # xvertseg MetaImage files
import sys
from sklearn.model_selection import train_test_split
import uuid

import matplotlib.pyplot as plt # data viz
import seaborn as sns # data viz

import imageio # display animated volumes
from IPython.display import Image # display animated volumes

from IPython.display import SVG # visualize model
from keras.utils.vis_utils import model_to_dot # visualize model

# for importing local code
src_dir = str(Path(projd.cwd_token_dir('notebooks')) / 'src') # $PROJECT_ROOT/src
if src_dir not in sys.path:
    sys.path.append(src_dir)

import util
import preprocessing
import datagen
import modelutil
import xvertseg
import augmentation


# MODEL_NAME = 'model_15'
SEED = 25 # random seed
EPOCHS = 100
BATCH_SIZE = 4
PATCH_SHAPE = (32, 32, 32)
VALIDATION_SPLIT = 0.25

DATA_DIR = Path('/data2').expanduser()
DATA_DIR = Path('~/data/2018').expanduser()
# UVMMC
NORMAL_SCANS_DIR = DATA_DIR / 'uvmmc/nifti_normals'
PROJECT_DATA_DIR = DATA_DIR / 'uvm_deep_learning_project'
PP_IMG_DIR = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed' # preprocessed scans dir
PP_MD_PATH = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed_metadata.pkl'
# xVertSeg
XVERTSEG_DIR = DATA_DIR / 'xVertSeg.v1'
PP_XVERTSEG_DIR = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed' # preprocessed scans dir
PP_XVERTSEG_MD_PATH = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed_metadata.pkl'


MODELS_DIR = PROJECT_DATA_DIR / 'models'
LOG_DIR = PROJECT_DATA_DIR / 'log'
TENSORBOARD_LOG_DIR = PROJECT_DATA_DIR / 'tensorboard'
TMP_DIR = DATA_DIR / 'tmp'

for d in [DATA_DIR, NORMAL_SCANS_DIR, PROJECT_DATA_DIR, PP_IMG_DIR, MODELS_DIR, LOG_DIR, 
          TENSORBOARD_LOG_DIR, TMP_DIR, PP_MD_PATH.parent, PP_XVERTSEG_DIR, PP_XVERTSEG_MD_PATH.parent]:
    if not d.exists():
        d.mkdir(parents=True)
        
%matplotlib inline
sns.set()

# I love u autoreload!
%load_ext autoreload
%autoreload 2

In [ ]:
infos_fun = preprocessing.make_preprocessed_metadata_fun(PP_XVERTSEG_MD_PATH)
df = pd.DataFrame({
    'id': range(1, 26),
    'pp_image_path': [Path(f'image{i:03}.mhd') for i in range(1, 26)],
    'pp_mask_path': [Path(f'mask{i:03}.mhd') for i in range(1, 16)] + ([np.nan] * 10),
})
infos_fun = lambda: df
train, val, test = xvertseg.get_xvertseg_datagens(
    infos_fun, validation_split=0.2, test_split=0.134)

In [ ]:
len(train)


In [ ]:
print(train[0])

In [ ]:
print(val[0])

In [ ]:
print(test[0])